# Simulating Hess Diagrams with StarFISH

In [1]:
%matplotlib inline
import os
import numpy as np
os.getenv("STARFISH")

'/Users/jsick/code/_starfish/starfish'

## Download isochrones

The [Padova package](http://github.com/jonathansick/padova) lets us quickly download a set of the Padova group's isochrones from the CMD website. Here we'll download a grid of isochrones to cover a set of WFC3 bands. (*Note: PHAT uses ACS for the optical bands*). Padova is able to export these isochrones into a format that StarFISH understands.

In [2]:
isoc_dirname = "padova_isoc"
full_isoc_dirname = os.path.join(os.getenv("STARFISH"),
                                 isoc_dirname)

wfc3_bands = ['F275W1', 'F336W', 'F475W', 'F814W', 'F110W', 'F160W']
# z_grid = [0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
#           0.001, 0.0012, 0.0016, 0.002, 0.0025, 0.0031, 0.0039, 0.0049, 0.0061, 0.0077, 0.0096,
#           0.012, 0.015, 0.019, 0.024, 0.03]
# z_grid = [0.015]
z_grid = [0.0096, 0.012, 0.015, 0.019, 0.024, 0.03]

In [3]:
import time
from padova import AgeGridRequest
for z in z_grid:
    r = AgeGridRequest(z, min_log_age=6.6, max_log_age=10.13, delta_log_age=0.05,
                       phot='wfc3', photsys_version='odfnew')
    for isoc in r.isochrone_set:
        isoc.export_for_starfish(full_isoc_dirname, bands=wfc3_bands)
    # time.sleep(10)

Reading from cache
Reading from cache
Reading from cache
Reading from cache
Reading from cache
Reading from cache


In [ ]:
from astropy.coordinates import Distance
import astropy.units as u

from starfisher import LibraryBuilder

input_dir = "wfc3_test"
lib_dir = "padova_lib"
d = Distance(785 * u.kpc)
builder = LibraryBuilder(input_dir, isoc_dirname, lib_dir,
                         nmag=len(wfc3_bands),
                         dmod=d.distmod.value,
                         iverb=3)
builder.install()

./mklib < wfc3_test/lib.dat


## Generating a synthetic Hess CMD library

In [ ]:
from starfisher import Lockfile, Synth, ExtinctionDistribution, MockNullCrowdingTable, ColorPlane

synth_dir = "wfc3_test/synth"
crowd_path = "wfc3_test/crowding.dat"

# Setup an isochrone locking scheme for 12 age bins
# Only use one metallicity
lockfile = Lockfile(builder, synth_dir)
lockfile.lock_grid(np.linspace(6.4, 10.5, num=12),
                   z_groups=[('0096', '0120'),
                             ('0150', '0190'),
                             ('0240', '0300')])

# No extinction
young_av = ExtinctionDistribution()
old_av = ExtinctionDistribution()
rel_extinction = np.ones(len(wfc3_bands), dtype=float)
for av in (young_av, old_av):
    av.set_uniform(0.)

# No photometric errors
crowd = MockNullCrowdingTable(crowd_path, len(wfc3_bands))

# Define colour planes to synthesize
opt_cmd = ColorPlane((wfc3_bands.index('F475W'), wfc3_bands.index('F814W')),
                     wfc3_bands.index('F814W'),
                     (-1., 3.),
                     (16., 30.),
                     35.,
                     suffix='f475f814',
                     x_label='F475W-F814W',
                     y_label='F814W')

synth = Synth(input_dir, builder, lockfile, crowd,
              rel_extinction, young_extinction=young_av, old_extinction=old_av,
              planes=[opt_cmd])
synth.run_synth(include_unlocked=False)
synth.plot_all_hess('synth_hess')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

from starfisher.plots import plot_isochrone_logage_logzsol, plot_lock_polygons

fig = plt.figure(figsize=(6.5, 4), dpi=300)
gs = mpl.gridspec.GridSpec(1, 1)
ax = fig.add_subplot(gs[0])

plot_isochrone_logage_logzsol(ax, builder, marker='.', s=20)
plot_lock_polygons(ax, lockfile, facecolor='None')

for gyr in range(1, 14):
    ax.axvline(np.log10(gyr * 1e9), c='b', alpha=0.5, zorder=-1)

ax.set_xlim(6., 11)
ax.set_ylim(-2, 0.5)
ax.set_xlabel(r"$\log(A)$")
ax.set_ylabel(r"$\log(Z/Z_\odot)$")
fig.show()

builder.isoc_logages

In [ ]:
for multipoly in lockfile.group_polygons:
    print "=="
    for verts in multipoly.logage_logzsol_verts:
        print verts